In [ ]:
import os
import re
import torch
import numpy as np
import pandas as pd

from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
device = torch.device('cuda')

#Loading the base model

In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import json

#Loading the QA data

In [ ]:
with open("/content/formatted_conversations.json", "r") as f:
    dataset = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/formatted_conversations.json'

In [ ]:
data_txt = []
# for i in range(speech_data.shape[0]):
#     o, p = speech_data.other_quote[i], speech_data.president_quote[i]
#     line = f'@@Q_START@@ {o} @@Q_END@@ @@A_START@@ {p} @@A_END@@'
#     data_txt.append(line)

for conversation in dataset:
    q, a = conversation['user'], conversation['system']
    line = f'@@Q_START@@ {q} @@Q_END@@ @@A_START@@ {a} @@A_END@@'
    data_txt.append(line)

In [ ]:
data_txt[:5]

['@@Q_START@@ What are the uses of a thru z advanced formula? @@Q_END@@ @@A_START@@ a thru z advanced formula is a multivitamin and iron product used to treat or prevent vitamin deficiency due to poor diet, certain illnesses, or during pregnancy. Vitamins and iron are important building blocks of the body and help keep you in good health. @@A_END@@',
 '@@Q_START@@ How to use a thru z advanced formula? @@Q_END@@ @@A_START@@ A Thru Z Advanced Formula Take a thru z advanced formula by mouth , usually once daily or as directed. Follow all directions on the product package, or take as directed by your doctor. Do not take more than the recommended dosage. If you have any questions, ask your doctor or pharmacist . a thru z advanced formula is best taken on an empty stomach 1 hour before or 2 hours after meals. Take with a full glass of water (8 ounces or 240 milliliters) unless your doctor directs you otherwise. If stomach upset occurs, you may take a thru z advanced formula with food. Avoid 

In [ ]:
with open('/content/drive/MyDrive/miniproject/data.txt', 'w') as w:
    for ex in data_txt:
        w.write(f'{ex}\n')

#Adding special tokens to the tokenizer to mark the question and answer

In [ ]:
# Adding the new tokens and saving the new tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_tokens(['@@Q_START@@', '@@Q_END@@', '@@A_START@@', '@@A_END@@'], special_tokens=True)


tokenizer.save_pretrained('/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer')

('/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer/vocab.json',
 '/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer/merges.txt',
 '/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer/added_tokens.json')

#Calling the GPT finetuning script with appropriate parameters

In [ ]:
!pip install datasets
!pip install evaluate
!pip install accelerate>=0.21.0
!python run_clm.py \
   --do_train \
   --model_name_or_path "gpt2" \
   --tokenizer_name "/content/drive/MyDrive/miniproject/new_tokenizer/new_tokenizer" \
   --model_type "gpt2" \
   --train_file "/content/drive/MyDrive/miniproject/data.txt" \
   --validation_file "/content/drive/MyDrive/miniproject/data.txt" \
   --cache_dir "/content/drive/MyDrive/miniproject/cache" \
   --output_dir "/content/drive/MyDrive/miniproject/logs_26042024" \
   --per_device_train_batch_size 2 \
   --gradient_accumulation_steps 8 \
   --save_total_limit 3 \
   --save_strategy "steps" \
   --eval_steps 500 \
   --save_steps 500 \
   --preprocessing_num_workers 4 \
   --num_train_epochs 3 \
   --block_size 512

2024-04-27 14:04:36.296707: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 14:04:36.296761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 14:04:36.298077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-27 14:04:37.499172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/27/2024 14:04:39 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprec

#Loading the checkpoint for inference

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/miniproject/logs_26042024'

model = GPT2LMHeadModel.from_pretrained(checkpoint_dir).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Memorize special tokens IDs
vocab = tokenizer.get_vocab()
id0, id1, id2, id3 = vocab['@@Q_START@@'], vocab['@@Q_END@@'], vocab['@@A_START@@'], vocab['@@A_END@@']
id0, id1, id2, id3

(50257, 50258, 50259, 50260)

#Inference on finetuned model

In [ ]:
input_text = ['What are the uses of baclofen?',
              'What are the precautions for rabies vaccine?',
              'What are side effects of b-complex?',
              'How to use bacitracin?',
              'What are the uses of calamine?',
              'What are the side effects of ear drops?',
              'What should be used for gas relief?',
              'What is interaction of pain relief medicine?',
              'What are the uses of valine?',
              'How to use vanacof?',
              'What are the side effects of xanax?']

for t in input_text:

    inputs = tokenizer(f'@@Q_START@@ {t} @@Q_END@@ @@A_START@@',
                       return_tensors='pt').to(device)

    generated = model.generate(**inputs,
                               top_p=0.7,
                               temperature=0.5,
                               do_sample=True,
                               eos_token_id=id3,
                               repetition_penalty=4.0,
                               max_length=100)

    generated = tokenizer.batch_decode(generated)[0]
    answer = re.search('@@A_START@@(.*)', generated).group(1)
    answer = re.sub('@@A_END@@', '', answer)

    print(f'QUESTION: {t}\nANSWER: {answer}', end='\n')

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: What are the uses of baclofen?
ANSWER:   BACLOFEN is used to treat a certain type ( chronic obstructive pulmonary disease ) that affects breathing and heart rhythm. It works by blocking an enzyme called angiotensin II receptor. The effect on blood flow may be temporary or permanent, depending upon how well you breathe, your condition/treatment response in combination with other medications such as antihistamines / antacids like ciprofloxifene ; alcohol; drugs for sleepiness -such among others). Some conditions can worsen if they do not get better within 2 weeks after treatment because symptoms disappear without warning from those who have them at least 3 months later than usual due either worsening coughs caused during therapy(including frequent chest pain ), difficulty sleeping while using medication it doesn't work right away but will last through high doses even when needed.(See also Side Effects section.) This drug passes into breast milk unless otherwise directed specifi

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: What are the precautions for rabies vaccine?
ANSWER:   Before taking rbv-15, tell your doctor or pharmacist if you have any new allergies. Rabie may contain inactive ingredients (such as soy products), which can cause allergic reactions. Talk to a health care professional about using lab tests safely and use reliable household items suchas toothbrushes, syringes/sprayers /rubbers because they work well together with saliva removal tablets that should be used only when clearly needed by mouth; especially during pregnancy unless directed otherwise in writing from an allergist before giving birth ; breastfeeding is not recommended while wearing this drug at all times since it increases risk of side effects including stomach upset caused due diarrhea lasting more than 2 days after stopping treatment without warning symptoms like vomiting ), difficulty urinating on day 7(8) but also unusual vaginal bleeding problems later into labor where there's no clear sign until 3 weeks postpa

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


...

QUESTION: What is interaction of pain relief medicine?
ANSWER:   Interaction with Pain Relief Medicine Does Palforzia interact or interfere in any other drugs you are taking, including prescription/nonprescription medicines, non-steroidal antiinflammatory (NSAID), and herbal products that may cause allergic reactions such as coffee grounds /flour. Ask your doctor if these substances can harm a child. If using pafarqan 0 mg oral powder tablet for asthma treatment on an outpatient basis because symptoms do not improve after 6 months due to the medication's use(s) it should be used only when clearly needed by treating severe breathing problems caused during therapy sessions; otherwise this drug could worsen those conditions even more severely than usual unless directed specifically so."


Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: What are the uses of valine?
ANSWER:   Valines is used to treat certain types (such as glaucoma, cancer ) that may be caused by a genetic disease. It works similar in different ways and can help prevent or shorten your period. If you have any questions, ask doctor/surgeon about using it safely for an extended time without causing serious side effects such bleeding problems with blood loss / swelling from chemotherapy treatments due at least 2 weeks before treatment begins - usually 3 months after stopping drugs like cetirizumab; if this occurs while taking other medications containing lamotrigene antibiotics-tetracyclin 1 billion cell suspension solution(s) including carbamazepam ; tamsulone acetate inhibitors prescribed under medical conditions involving high fever symptoms lasting more than 24 hours unless otherwise directed on label directions within 48 hr following dose withdrawal because they do not work well when taken together during pregnancy). This drug passes into b

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: How to use vanacof?
ANSWER:   VanAcOxygen is used as a natural supplement for the treatment of certain skin conditions (such eczema, psoriasis ). It works by reducing itching and irritation caused when you apply it. If your condition lasts or gets worse, get medical help right away if symptoms worsen. To decrease swelling/redness in those areas where they may be most likely affected: underarms /feet area; groin sore throat ; eye problems such acne scars from sun exposure that need protection before falling asleep(s). Use moisturizer regularly after applying this drug because dry patches can become discolored during drying out periods while using these products - especially at nighttime due not wearing protective clothing on time-use days like hot weather day 7 through 12 hours unless directed otherwise within 48 hr ahead notice with instructions regarding how long each dose should stay inside an appropriate container so there are no more red spots until later than 6 weeks old

#Inference on base model for comparison

In [ ]:
base_model = GPT2LMHeadModel.from_pretrained("gpt2")
base_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
#base_tokenizer.add_tokens(['@@Q_START@@', '@@Q_END@@', '@@A_START@@', '@@A_END@@'], special_tokens=True)

In [ ]:
input_text = ['What are the uses of baclofen?',
              'What are the precautions for rabies vaccine?',
              'What are side effects of b-complex?',
              'How to use bacitracin?',
              'What are the uses of calamine?',
              'What are the side effects of ear drops?',
              'What should be used for gas relief?',
              'What is interaction of pain relief medicine?',
              'What are the uses of valine?',
              'How to use vanacof?',
              'What are the side effects of xanax?']

for t in input_text:

    inputs = base_tokenizer(t,
                       return_tensors='pt')

    generated = base_model.generate(**inputs,
                               top_p=0.7,
                               temperature=0.5,
                               do_sample=True,
                               eos_token_id=id3,
                               repetition_penalty=4.0,
                               max_length=200)

    generated = base_tokenizer.batch_decode(generated)[0]
    # answer = re.search('@@A_START@@(.*)', generated).group(1)
    # answer = re.sub('@@A_END@@', '', answer)

    print(generated)

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


What are the uses of baclofen?
A. Bacterial infections, such as those caused by bacteria or viruses that cause foodborne illness and can be transmitted to humans through contact with contaminated foods (such a person's own body), may result in serious health problems for some people who have been exposed but not others because they were infected from other sources including: exposure due directly into an airway; contamination resulting from being caught on camera during filming at home where there is no security risk associated With all these factors considered it should come down to whether you would consider using this product if your life depended upon its use over time A good safety record will help ensure any potential risks do occur The following products contain active ingredients which could pose significant hazards when used together In addition we recommend caution against handling them while under close supervision If taken without warning please take precautions before taki

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


What are the precautions for rabies vaccine?
1. Vaccine safety and effectiveness: The best way to protect against a disease is by taking care of your own health, not others' or anyone else's needs (e-mail). You should be able talk with someone who has been vaccinated before you have any symptoms; it may take time but if they know what their condition means in terms that will help them make informed decisions about whether to get an injection at home instead as opposed on other days when there might otherwise still be complications from infection such like rash etc., then this can save some lives! 2) Use only safe vaccines – no one wants another person getting sick because he/she was exposed through exposure too much…it doesn't matter how good his immune system reacts so long ago unless we're talking more than just physical contact between people which could lead us into serious problems later down line 3.) Don´t give up hope until after vaccination - once all vaccinations go out I beli

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


...

What is interaction of pain relief medicine?
The most common way to treat a chronic disease such as arthritis or heart failure, which can cause severe and debilitating symptoms like joint stiffness. The best treatment for these conditions are the traditional methods that have been used in many countries since ancient times: acupuncture (the practice of using needles on your joints), massage therapy (using an electric current through one's body) etc., but this method has its limitations because it requires you be able "to feel" something without having any experience with other types; there isn't much information available about how effective they actually do when applied properly so I won´t go into all details here! However if we look at some specific cases where people were treated successfully by conventional medical care then perhaps our understanding would change somewhat…

 There was also another case from India called 'Kamal' who had received his first injection after being diagn

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


What are the uses of valine?
Avalines have a wide variety in their use, and many people find them useful for various purposes. The following list is intended to help you decide which ones will best suit your needs:<|endoftext|>The first time I saw this movie was at my local theater when it came out on DVD back then! It's one helluva thing that they did with "Fantastic Four" (the original film) but what really makes me think about these guys who made such an amazing set-up so far from all those other movies like Avatar or Star Wars...it just seems too much fun without being real good enough!! And now we're getting into some new stuff as well!!!

 I'm not going anywhere soon though because there isn't anything here yet - even if everyone else has seen/heard something before....but wait until next week where things get interesting.....and maybe see more pics :) __________________ Last edited by Dandy; 08th


Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


How to use vanacof?
The best way is by using a standard filter. A typical method for filtering out the unwanted noise in your car's exhaust system would be with an air compressor, but this will not work if you have one that has been used before and it does NOT need any special equipment (such as filters). You can also make some sort of "filter" from carbon fiber or other materials such Asbestos-free material like wood chips etc., which are very effective at reducing emissions when they come into contact directly on surfaces where there may already exist asbestos dust particles inside them:

<|endoftext|>A few weeks ago I wrote about how much better we could do without Apple Pay than Google Wallet because now all payments made through our mobile apps require us access tokens instead! This makes sense since many people don't even know what their credit card number actually looks so most users just assume something else might look different depending upon whether someone uses NFC tags whi

#Perplexity Score evalutaion of finetuned model

In [ ]:
# import torch
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

def evaluate_perplexity(model, tokenizer, text):
    # Add special tokens to the text
    text_with_special_tokens = f'@@Q_START@@ {text} @@Q_END@@ @@A_START@@'

    # Tokenize the text
    input_ids = tokenizer.encode(text_with_special_tokens, return_tensors='pt').to(device)

    # Generate logits from the model
    with torch.no_grad():
        logits = model(input_ids)[0]

    # Compute softmax probabilities
    softmax_logits = torch.softmax(logits, dim=-1)[0]

    # Remove special tokens from input_ids and convert to list
    token_ids = input_ids[0, 1:].squeeze().tolist()

    # Compute log probabilities for the actual token ids
    log_probs = torch.log(softmax_logits[range(len(token_ids)), token_ids])

    # Compute perplexity
    perplexity = torch.exp(-torch.sum(log_probs) / log_probs.shape[-1]).item()

    return perplexity


# Load fine-tuned GPT-2 model and tokenizer
# model_name = "your-finetuned-model-name"
# model = GPT2LMHeadModel.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Example text for evaluation
input_text = ['What are the uses of baclofen?',
              'What are the precautions for rabies vaccine?',
              'What are side effects of b-complex?',
              'How to use bacitracin?',
              'What are the uses of calamine?',
              'What are the side effects of ear drops?',
              'What should be used for gas relief?',
              'What is interaction of pain relief medicine?',
              'What are the uses of valine?',
              'How to use vanacof?',
              'What are the side effects of xanax?']
# model = GPT2LMHeadModel.from_pretrained(checkpoint_dir)
# tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_dir)
# Evaluate perplexity
for t in input_text:
    perplexity = evaluate_perplexity(model, tokenizer, t)
    print("Perplexity:", perplexity)

Perplexity: 3.9826791286468506
Perplexity: 11.150195121765137
Perplexity: 14.743796348571777
Perplexity: 5.1560139656066895
Perplexity: 6.037504196166992
Perplexity: 10.14511775970459
Perplexity: 30.17329216003418
Perplexity: 41.22710418701172
Perplexity: 6.888851165771484
Perplexity: 16.597122192382812
Perplexity: 7.372170448303223


#Perplexity Score evaluation for base model

In [ ]:
def evaluate_perplexity(model, tokenizer, text):
    # Add special tokens to the text
    text_with_special_tokens = f'@@Q_START@@ {text} @@Q_END@@ @@A_START@@'

    # Tokenize the text
    input_ids = tokenizer.encode(text_with_special_tokens, return_tensors='pt')

    # Generate logits from the model
    with torch.no_grad():
        logits = model(input_ids)[0]

    # Compute softmax probabilities
    softmax_logits = torch.softmax(logits, dim=-1)[0]

    # Remove special tokens from input_ids and convert to list
    token_ids = input_ids[0, 1:].squeeze().tolist()

    # Compute log probabilities for the actual token ids
    log_probs = torch.log(softmax_logits[range(len(token_ids)), token_ids])

    # Compute perplexity
    perplexity = torch.exp(-torch.sum(log_probs) / log_probs.shape[-1]).item()

    return perplexity

for t in input_text:
    perplexity = evaluate_perplexity(base_model, base_tokenizer, t)
    print("Perplexity:", perplexity)

Perplexity: 50.25851821899414
Perplexity: 56.80854034423828
Perplexity: 55.16973876953125
Perplexity: 83.84107971191406
Perplexity: 63.5296630859375
Perplexity: 49.16680145263672
Perplexity: 65.45885467529297
Perplexity: 102.52810668945312
Perplexity: 59.03614044189453
Perplexity: 115.41448974609375
Perplexity: 40.51702117919922
